In [9]:
################################################## ITERATIVE CLOSEST POINT ALGORITHM ######################################################################
# REQUIRED FUNCTIONS
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

def EstimateCorrespondences(X,Y,R,t,dmax):
    correlations = []
    for i in range(X.shape[0]):
        xi = X[i,:].reshape(3,1)
        differences = np.transpose(Y)-(R@xi)-t    # 3x5000 matrix
        distances  = np.linalg.norm(differences,axis = 0)
        # print(distances) 
        j = np.argmin(distances)    # Gives the index of least distance for that Xi
        if distances[j]<dmax:
            correlations.append([i,j])   # Storing correlation as X first and Y second.
    correlations = np.array(correlations) 
    return correlations, correlations.shape[0]

def ComputeOptimalRigidRegistration(X,Y,C,K): 
    dev_y = np.zeros((3,K))
    dev_x = np.zeros((3,K))
    centroid_X = np.zeros((3))
    centroid_Y = np.zeros((3))
    for k in range(K):
        centroid_X = centroid_X + X[C[k,0],:]/K
        centroid_Y = centroid_Y + Y[C[k,1],:]/K   #  3, matrix i.e., 1D matrix 
    for k in range(K):    #dev_x[:,k] is a 1D matrix. 
        dev_x[:,k] = X[C[k,0],:] - centroid_X
        dev_y[:,k] = Y[C[k,1],:] - centroid_Y    #3x1
      
    W = (1/K) * (dev_y@dev_x.T) 
    
    U,S,VT = np.linalg.svd(W)
    R_cap = U @ VT
    t_cap = centroid_Y.reshape(3, 1) - R_cap@centroid_X.reshape(3, 1)
    return R_cap,t_cap     #t is a 3x1 matrix

def ICP(X,Y,R0,t0,dmax,num_ICP_iters):    #t0 is a 3x1 matrix
    t_cap = t0
    R_cap = R0    # Initialization of R,t
    for n in range(num_ICP_iters):
        C,K = EstimateCorrespondences(X,Y,R_cap,t_cap,dmax)
        R_cap,t_cap = ComputeOptimalRigidRegistration(X,Y,C,K)
    return R_cap,t_cap,C,K

In [10]:
 # EXECUTION ON POINT CLOUDS

X = np.loadtxt('pclX.txt')
Y = np.loadtxt('pclY.txt')
dmax = 0.25
R0 = np.eye(3)
t0 = np.array([0,0,0]).reshape(3, 1)
num_ICP_iters= 30 

R,t,C,K = ICP(X,Y,R0,t0,dmax,num_ICP_iters)
X_transformed = np.transpose(R@np.transpose(X)+t)  # Nx3
print("The rotation from ICP is \n")
print(R)
print("The translation from ICP is \n")
print(t)

The rotation from ICP is 

[[ 0.95126601 -0.15043058 -0.26919069]
 [ 0.22323628  0.9381636   0.26460276]
 [ 0.21274056 -0.31180074  0.92602471]]
The translation from ICP is 

[[ 0.49661487]
 [-0.29392971]
 [ 0.29645004]]


In [11]:
# PLOTTING
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Y[:,0], y=Y[:,1], z=Y[:,2], mode='markers', name='Y Pointcloud',marker=dict(size=3)))
fig.add_trace(go.Scatter3d(x=X_transformed[:,0], y=X_transformed[:,1], z=X_transformed[:,2], mode='markers', name='Transformed X Pointcloud', marker=dict(color='red',size = 3)))
fig.add_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2], mode='markers', name='X Pointcloud', marker=dict(color='green',size = 3)))
fig.show()

In [12]:
# RMSE CALCULATION USING C

difference = np.zeros((3,K), dtype=float) 
Y_c = Y[C[:,1],:]
X_c = X[C[:,0],:]     # Corresponding points from the final ICP iteration. X_c and Y_c are Kx3 arrays. 
difference = Y_c.T - R@X_c.T - t
square_norms = np.square(np.linalg.norm(difference,axis = 0))
MSE = square_norms.sum()/K
RMSE = np.sqrt(MSE)
print("RMSE is as follows",RMSE)


RMSE is as follows 0.008950576587683123
